# Curate Data
> Make the data really good

## Fixing Bad Data

The prompt looks like this:

```
....

You are given the following four inputs: (1) NLQ, (2) A list of candidate columns that are allowed to be in the query (3) The query and (4) a critique. Your goal is to improve the query based on the critique suggests you should do so.
NLQ: name contains  funtikpub-processTask-30874598
    
COLUMNS: ['articleunpublished.data.id', 'name', 'service.name', 'telemetry.sdk.name', 'app.newDBTweet.UserScreenName', 'host.name', 'app.dbTweet.Tweet.PublishedAt', 'library.name', 'feedItemID', 'app.dbTweet.Tweet.UserScreenName', 'articleenriched.data.id', 'app.dbTweet.Tweet.ExternalID', 'app.buildMeta.Title', 'net.host.name', 'error', 'app.dbTweet.Tweet.ID', 'app.user.Name', 'service.version', 'app.user.Username', 'app.newDBTweet.UserName', 'newsItemID', 'event.data.id', 'app.buildMeta.PublishTime', 'was_tagged', 'app.dbTweet.Tweet.Text', 'span.kind', 'tweet_id', 'app.dbTweet.Tweet.UserName', 'app.user.Verified', 'app.newDBTweet.ExternalID', 'event.metadata.event_id', 'app.user.Protected', 'feedItem.ID', 'app.dbTweet.Tweet.UserImage', 'app.newDBTweet.PublishedAt', 'feedItem.ItemID', 'app.apiTweet.User.ScreenName', 'app.dbTweet.Tweet.UserURL', 'trace.trace_id', 'app.buildMeta.Teaser', 'http.flavor', 'app.apiTweet.User.Name', 'app.loadTweet.tweet', 'parent_name', 'notification.FeedItemStream.Id', 'app.newDBTweet.ID', 'honeycomb.distro.runtime_version', 'app.v2Tweet.Data.Id', 'app.dbTweet.Tweet.UserVerified', 'app.buildMeta.Media.Images', 'http.route', 'trace.parent_id', 'duration_ms', 'http.status_code', 'exception.message']

QUERY: {"breakdowns":["name"],"calculations":[{"op":"COUNT"}],"filters":[{"column":"name","op":"contains","value":"funtikpub-processTask-30874598"}],"time_range":7200}

CRITIQUE: This query is good because it uses the 'contains' operator to filter by a substring of 'name', which is exactly what is needed based on the NLQ, 'name contains funtikpub-processTask-30874598'. The query also correctly applies a COUNT calculation, which gives the number of instances that match the filter criteria, and breaks down the results by 'name'. This should provide insights into the frequency of occurrences of each unique name containing 'funtikpub-processTask-30874598' within the last 2 hours, as specified by the 'time_range'.

Output the improved query and nothing else in a json format adhereing to the QUERY SPEC.
```

## Filtering Data

1. Use level one eval logic to filter invalid queries
2. Use level two eval logic to filter queries (you can also try to heal)
3. Apply other kinds of filters - in this case filtered out queries that were (1) too simple or too complex (2) near duplicates.

```python
def complexity(q):
    "calculate complexity score for query."
    l1_keys = len(q)
    l2_keys = 0
    l2_vals = 0
    
    for k in q:
        val = q[k]
        if isinstance(val, dict):
            l2_keys += len(val)
        elif isinstance(val, list):
            cnt = sum([len(l) if isinstance(l, dict) else 1 for l in val])
            if cnt == 0: return 0 # so we can filter out queries with empty values
            else: l2_vals += cnt
    return l1_keys + l2_keys + l2_vals
```

### Lilac

A popular tool for searching for, filtering, etc duplicates is [Lilac](https://www.lilacml.com/).


### A naive way to filter near duplicates

This is very specific to this use case. Start simple.

We want to filter duplicates where the (`nlq`, `cols`), (`nlq`, `query`), or (`cols`, `query`) is the same.

```python
valid_synth_df = (valid_synth_df
 .drop_duplicates(subset=['nlq', 'col_set'])
 .drop_duplicates(subset=['col_set', 'str_query'])
 .drop_duplicates(subset=['nlq', 'str_query'])
)
```

